In [2]:
import numpy as np
import itertools
import picos as pic
import Fun as f
import mosek
import sys

In [13]:
def SDP(n,K,d,D):
    
    ##### SDP #####

    pic.available_solvers()
    
    Ap = pic.RealVariable("Ap", n+1)  # A column vector with n+1 elements.
    
    sdp = pic.Problem()
    
    #sdp.options["abs_*_tol"] = 1e-05
    
    sdp.set_objective('find') 
                   
    ##### Adding matrix constraints #####
    
    sdp.add_constraint(Ap[0]==1/K**3)
    
    ##### NPA #####
    
    eps=1/K**3
    
    eps_terw=1/K**3
    
    ### Constructing Beta matrices ###
    
    Bmatrix=f.BetaMatrices(Ap,n) #### B_j(x)>>0 ####
 
    sdp.add_list_of_constraints(eps_terw*pic.block(Bmatrix[k])>>0 for k in range(int(n/2)+1))
        
    ##### Adding K-L constrains #####
    
    ### K*B'_j-A'_j=0  for d > j >= 0 ###
    
    sdp.add_list_of_constraints((K*Ap[n-i]-Ap[i])== 0 for i in range(d)) 
    
    ### A_j>=0 ###
    
    for m in range(n+1):
        A=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*Ap[j] for j in range(m+1))
        sdp.add_constraint(A*eps >= 0)

    ### K*B_j-A_j>=0 ###
    
    for m in range(n+1):   
                                    
        KBA=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*(K*Ap[n-j]-Ap[j]) for j in range(m+1))
        sdp.add_constraint(KBA*eps >= 0)
    
    ##### Adding Shadow constrains #####
    
    ### sum(K_j*A'_j)>=0 ###
    
    for j in range(n+1):
        
        S=sum(f.K_fun(n-j,i,n)*Ap[i] for i in range(n+1))
        #sdp.add_constraint(S*eps >= 0)
        
    ### Convention for K=1 codes ###
    
    if K==1:
        
        sdp.add_list_of_constraints(Ap[i]== f.comb(n,i)/D**i for i in range(1,d))

    sol = sdp.solve(primals=False)
    
    if sol.problemStatus == "feasible":
        
        return True    
    
    elif sol.problemStatus == "infeasible":

        #task=sdp.strategy.solver.int
        #task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
        #print(task.solutionsummary(mosek.streamtype.msg))
        
        return False
    else:
        return sol.problemStatus 

In [3]:
"""
exec(open("./qutrit_upper_bounds.py").read())

n_min=19
n_max=20

D=3

for n in range(n_min,n_max+1):
    
    k=0
    
    for d in L_upper_qutrit[n-1]:
        
        
        #print("[%s,%s,%s]" % (n,k,d))
        
        check=SDP(n,D**k,d,D)
        
        
        
        if check != True:
            print("[%s,%s,%s]=%s" % (n,k,d,check))
        
        k += 1
print("Done")
"""

'\nexec(open("./qutrit_upper_bounds.py").read())\n\nn_min=19\nn_max=20\n\nD=3\n\nfor n in range(n_min,n_max+1):\n    \n    k=0\n    \n    for d in L_upper_qutrit[n-1]:\n        \n        \n        #print("[%s,%s,%s]" % (n,k,d))\n        \n        check=SDP(n,D**k,d,D)\n        \n        \n        \n        if check != True:\n            print("[%s,%s,%s]=%s" % (n,k,d,check))\n        \n        k += 1\nprint("Done")\n'

In [4]:

exec(open("./qutrit_upper_bounds.py").read())


n_min=19
n_max=20

D=3

for n in range(n_min,n_max+1):
    
    k=0
    
    for d in L_upper_qutrit[n-1]:
        
        check=SDP(n,D**k,d+1,D)
        
        if check != False:
            print("[%s,%s,%s]=%s" % (n,k,d+1,check))
        
        k += 1
print("Done")


[19,2,9]=unknown
[19,3,9]=unknown
[19,4,8]=True


KeyboardInterrupt: 

In [15]:
D=3

Codes=[[11,0,6]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],D**codes[1],codes[2],D)))

[11, 0, 6]=True
